# Simple chat response for all rows in CSV

This notebook will read your CSV, prompt and for each row in the CSV will ask gpt-4 for a response that will be recorded in the output

In [ ]:
import os
import datetime

import pandas as pd
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import openai

In [ ]:
# Input paramters
INPUT_CSV = "../workspace/buildup_christianity_usa/tweets.csv"
COLUMNS_FOR_ANALYSIS = ["text"]
OUTPUT_CSV = "../workspace/buildup_christianity_usa/output.csv"
PROMPT_FILE = "../workspace/buildup_christianity_usa/prompt.txt"
START_OFFSET = 0
NUMBER_OF_RECORDS_TO_PROCESS = 1
API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
input_df = pd.read_csv(INPUT_CSV)
input_df.head()

In [ ]:
series_of_text = input_df[COLUMNS_FOR_ANALYSIS].apply(' '.join, axis=1)
series_of_text.head()

In [ ]:
if NUMBER_OF_RECORDS_TO_PROCESS is None:
    series_to_process = series_of_text[START_OFFSET:]
else:
    series_to_process = series_of_text[START_OFFSET:NUMBER_OF_RECORDS_TO_PROCESS]

print(f"total number of records to process is {series_to_process.shape[0]}")

In [ ]:
with open(PROMPT_FILE) as f:
    prompt = f.read()

prompt

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(prompt)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [ ]:
chatgpt_responses = series_to_process.apply(lambda x: chain.run(text=x))

In [ ]:
chatgpt_responses.head()

In [ ]:
output_df = input_df.copy()
output_df["gpt_response"] = chatgpt_responses
output_df["gpt_responsed_at"] = datetime.datetime.now(datetime.timezone.utc)

In [ ]:
output_df.head()

In [ ]:
output_df.to_csv(OUTPUT_CSV)